In [ ]:
import sys
import os

notebook_path = os.getcwd() 
parent_dir = os.path.dirname(notebook_path)
project_root = os.path.dirname(parent_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from copy import deepcopy

In [ ]:
import state_NN_models
import Filters
import utils
import Systems
from utils import losses, trainer, utils
from torch.utils.data import TensorDataset, DataLoader, random_split
from state_NN_models.StateBayesianKalmanNet import StateBayesianKalmanNet
from state_NN_models.StateKalmanNet import StateKalmanNet
from state_NN_models.StateKalmanNetWithKnownR import StateKalmanNetWithKnownR

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

In [ ]:
import torch
from math import pi

# =================================================================================
# KROK 1: DEFINICE PARAMETRŮ PRO NELINEÁRNÍ SYSTÉM ("Synthetic")
# =================================================================================

state_dim_nl = 2
obs_dim_nl = 2

# --- Parametry reálného systému ("Ground Truth") ---
# Tyto hodnoty odpovídají 'Full' knowledge v jejich kódu
alpha_true = 0.9
beta_true = 1.1
phi_true = 0.1 * pi
delta_true = 0.01
a_true = 1.0
b_true = 1.0
c_true = 0.0

# Definice nelineárních funkcí pomocí lambda
# Důležité: Musí být schopné pracovat s dávkami (batches)!
f_true_nonlinear = lambda x: alpha_true * torch.sin(beta_true * x + phi_true) + delta_true
h_true_nonlinear = lambda x: a_true * (b_true * x + c_true)**2

# Parametry šumu a počátečních podmínek
Q_true_nl = torch.eye(state_dim_nl) * 0.5  # Šum procesu
R_true_nl = torch.eye(obs_dim_nl) * 0.1  # Šum měření
Ex0_true_nl = torch.tensor([[1.0], [0.0]])
P0_true_nl = torch.eye(state_dim_nl) * 1.5


# --- Parametry modelu systému ("Partial Knowledge") ---
# Tyto hodnoty odpovídají 'Partial' knowledge, simulují nepřesný model
alpha_model = 1.0
beta_model = 1.0
phi_model = 0.0
delta_model = 0.0
a_model = 1.0
b_model = 1.0
c_model = 0.0

# Definice nelineárních funkcí modelu
f_model_nonlinear = lambda x: alpha_model * torch.sin(beta_model * x + phi_model) + delta_model
h_model_nonlinear = lambda x: a_model * (b_model * x + c_model)**2

# Model může mít i nepřesnou znalost šumu a počátečních podmínek
Q_model_nl = torch.eye(state_dim_nl) * 0.1
R_model_nl = R_true_nl # Předpokládejme, že R známe přesně
Ex0_model_nl = torch.tensor([[0.5], [0.5]])
P0_model_nl = torch.eye(state_dim_nl) * 1.0


# =================================================================================
# KROK 2: INICIALIZACE OBJEKTŮ SYSTÉMŮ
# =================================================================================
# Ujisti se, že proměnná `device` je definována
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("\nInicializuji 2D 'Synthetic' nelineární systém...")

# Reálný systém, který bude generovat data
sys_true = Systems.DynamicSystem(
    state_dim=state_dim_nl, obs_dim=obs_dim_nl,
    Ex0=Ex0_true_nl, P0=P0_true_nl,
    Q=Q_true_nl, R=R_true_nl,
    f=f_true_nonlinear, h=h_true_nonlinear, # Předáváme funkce, ne matice
    device=device
)

# Model, který bude používat tvůj KalmanNet (s nepřesnými parametry)
sys_model = Systems.DynamicSystem(
    state_dim=state_dim_nl, obs_dim=obs_dim_nl,
    Ex0=Ex0_model_nl, P0=P0_model_nl,
    Q=Q_model_nl, R=R_model_nl,
    f=f_model_nonlinear, h=h_model_nonlinear, # Předáváme funkce, ne matice
    device=device
)

print("... Nelineární systém inicializován.")

In [ ]:
TRAIN_SEQ_LEN = 100      # Krátké sekvence pro stabilní trénink (TBPTT)
VALID_SEQ_LEN = 200      # Stejná délka pro konzistentní validaci
TEST_SEQ_LEN = 100      # Dlouhé sekvence pro testování generalizace

NUM_TRAIN_TRAJ = 50   # Hodně trénovacích příkladů
NUM_VALID_TRAJ = 10    # Dostatek pro spolehlivou validaci
NUM_TEST_TRAJ = 100     # Pro robustní vyhodnocení

BATCH_SIZE = 256         # Dobrý kompromis

x_train, y_train = utils.generate_data(sys_true, num_trajectories=NUM_TRAIN_TRAJ, seq_len=TRAIN_SEQ_LEN)
x_val, y_val = utils.generate_data(sys_true, num_trajectories=NUM_VALID_TRAJ, seq_len=VALID_SEQ_LEN)
x_test, y_test = utils.generate_data(sys_true, num_trajectories=1, seq_len=TEST_SEQ_LEN)

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# import torch
# import torch.nn as nn
# from torch.utils.data import TensorDataset, DataLoader
# import numpy as np
# import os
# import random
# import csv
# from datetime import datetime
# import pandas as pd
# from copy import deepcopy

# model_config = {
#     "hidden_size_multiplier": 10,
#     "output_layer_multiplier": 4,
#     "num_gru_layers": 1,
#     "init_min_dropout": 0.5,
#     "init_max_dropout": 0.8
# }

# train_config = {
#     "total_train_iter": 1200,
#     "learning_rate": 1e-4,
#     "clip_grad": 10.0,
#     "J_samples": 20,
#     "validation_period": 20,
#     "logging_period": 20,
#     "warmup_iterations":0 # Trénuj prvních 400 iterací jen na MSE
# }

# # =================================================================================
# # KROK 3: SPUŠTĚNÍ JEDNOHO TRÉNINKOVÉHO BĚHU
# # =================================================================================

# print("="*80)
# print("Spouštím jeden plnohodnotný tréninkový běh...")
# print(f"Parametry modelu: {model_config}")
# print(f"Parametry tréninku: {train_config}")
# print("="*80)

# # Nastavení seedu pro reprodukovatelnost tohoto běhu
# torch.manual_seed(42)
# np.random.seed(42)
# random.seed(42)

# # Vytvoření modelu
# state_bkn_knet = StateBayesianKalmanNet(
#     sys_model,
#     device=device,
#     **model_config
# ).to(device)

# # Spuštění tréninku
# # Používáme `run_training_session`, která vrací slovník s výsledky
# results = trainer.training_session_trajectory_with_gaussian_nll_training_fcn(model=state_bkn_knet,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     device=device,
#     **train_config
# )

# # `run_training_session` automaticky načte nejlepší model zpět,
# # takže `state_bkn_knet` nyní obsahuje váhy nejlepšího modelu.
# trained_model = results['final_model']

# print("\n" + "="*80)
# print("TRÉNINK DOKONČEN - FINÁLNÍ VÝSLEDKY Z NEJLEPŠÍHO MODELU")
# print("="*80)
# print(f"Nejlepší model byl nalezen v iteraci: {results['best_iter']}")
# # --- Změněné klíče, aby odpovídaly return statementu ---
# print(f"Nejlepší dosažený validační ANEES: {results['best_val_anees']:.4f}")
# print("--- Metriky odpovídající tomuto nejlepšímu modelu ---")
# print(f"  MSE na validační sadě:       {results['best_val_mse']:.4f}")
# print(f"  NLL na validační sadě:       {results['best_val_nll']:.4f}")
# print("="*80)

# # Nyní můžeš s `trained_model` pokračovat, například ho vyhodnotit na testovací sadě.

In [ ]:

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import random
import csv
from datetime import datetime
import pandas as pd
from copy import deepcopy

from state_NN_models import StateKalmanNet_v2

# Nastavení seedu pro reprodukovatelnost tohoto běhu
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
state_knet = StateKalmanNet_v2(sys_model, device=device, hidden_size_multiplier=8,output_layer_multiplier=4,num_gru_layers=1,gru_hidden_dim_multiplier=4).to(device)
trainer.train_state_KalmanNet_sliding_window(
    model=state_knet, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    device=device, 
    epochs=200, 
    lr=1e-3,
    early_stopping_patience=30,
    tbptt_k=2,
    tbptt_w=8,
    optimizer_=torch.optim.AdamW,
    weight_decay_=1e-3
)

In [9]:

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import random
import csv
from datetime import datetime
import pandas as pd
from copy import deepcopy
# Nastavení seedu pro reprodukovatelnost tohoto běhu
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
state_knetR = StateKalmanNetWithKnownR(sys_model, device=device, hidden_size_multiplier=12).to(device)
trainer.train_state_KalmanNet(
    model=state_knetR, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    device=device, 
    epochs=100, 
    lr=1e-3,
    early_stopping_patience=20
)

Epoch [5/100], Train Loss: 0.736695, Val Loss: 0.765070, Avg Cov Trace: 0.027099
Epoch [10/100], Train Loss: 0.714183, Val Loss: 0.714035, Avg Cov Trace: 0.070767
Nové nejlepší model uloženo! Epoch [10/100], Train Loss: 0.714183, Val Loss: 0.714035
Nové nejlepší model uloženo! Epoch [11/100], Train Loss: 0.701171, Val Loss: 0.700669
Nové nejlepší model uloženo! Epoch [12/100], Train Loss: 0.686158, Val Loss: 0.689013
Nové nejlepší model uloženo! Epoch [13/100], Train Loss: 0.671628, Val Loss: 0.680718
Nové nejlepší model uloženo! Epoch [14/100], Train Loss: 0.662204, Val Loss: 0.674200
Epoch [15/100], Train Loss: 0.656624, Val Loss: 0.669983, Avg Cov Trace: 0.066270
Nové nejlepší model uloženo! Epoch [15/100], Train Loss: 0.656624, Val Loss: 0.669983
Nové nejlepší model uloženo! Epoch [16/100], Train Loss: 0.652167, Val Loss: 0.666300
Nové nejlepší model uloženo! Epoch [17/100], Train Loss: 0.647525, Val Loss: 0.663036
Nové nejlepší model uloženo! Epoch [18/100], Train Loss: 0.643241, 

StateKalmanNetWithKnownR(
  (dnn): DNN_KalmanNet(
    (input_layer): Linear(in_features=4, out_features=96, bias=True)
    (gru): GRU(96, 96)
    (output_layer): Linear(in_features=96, out_features=4, bias=True)
  )
)

In [10]:
from state_NN_models import KalmanFormer
model = KalmanFormer(
    sys_model, 
    device,
    d_model=32,       # Hyperparametry Transformeru
    nhead=4,          # Hyperparametry Transformeru
    num_encoder_layers=2,
    num_decoder_layers=2,
    dim_feedforward=64
).to(device)

# Trénovací funkce se volá úplně stejně:
trained_model_former = trainer.train_state_KalmanNet_sliding_window(
    model, 
    train_loader, 
    val_loader, 
    device,
    epochs=200,
    lr=1e-3, # Možná bude potřeba jiná LR!
    tbptt_k=2,
    tbptt_w=8
)

INFO: Detekováno z atributu modelu, že vrací kovarianci: False
INFO: Spouštím trénink s TBPTT(k=2, w=8)
Nové nejlepší model uloženo! Epoch [1/200], Train Loss: 3.955826, Val Loss: 1.352112
Nové nejlepší model uloženo! Epoch [2/200], Train Loss: 2.342866, Val Loss: 0.887160
Nové nejlepší model uloženo! Epoch [3/200], Train Loss: 1.780448, Val Loss: 0.790597
Epoch [5/200], Train Loss: 1.500163, Val Loss: 0.760662
Nové nejlepší model uloženo! Epoch [5/200], Train Loss: 1.500163, Val Loss: 0.760662
Nové nejlepší model uloženo! Epoch [7/200], Train Loss: 1.317226, Val Loss: 0.743440
Nové nejlepší model uloženo! Epoch [8/200], Train Loss: 1.235927, Val Loss: 0.732656
Epoch [10/200], Train Loss: 1.139303, Val Loss: 0.757543
Nové nejlepší model uloženo! Epoch [11/200], Train Loss: 1.118374, Val Loss: 0.703914
Nové nejlepší model uloženo! Epoch [14/200], Train Loss: 1.027168, Val Loss: 0.690780
Epoch [15/200], Train Loss: 1.004703, Val Loss: 0.699412
Nové nejlepší model uloženo! Epoch [18/200],

In [11]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# ==============================================================================
# 0. PŘEDPOKLADY - ZDE PŘIŘAĎTE VAŠE NATRÉNOVANÉ MODELY
# ==============================================================================
# Ujistěte se, že v proměnných níže máte již natrénované a připravené modely.
# Názvy proměnných si upravte podle vašeho kódu, pokud se liší.
try:
    # trained_model_bkn = trained_model
    trained_model_classic = state_knet
    trained_model_knetR = state_knetR
    
    # Ujistěte se, že zde máte přiřazený i natrénovaný KalmanFormer
    trained_model_former = trained_model_former # <--- PŘIDÁNO (nebo váš název proměnné)
    
    print("INFO: Všechny natrénované modely nalezeny a přiřazeny.")
except NameError as e:
    print(f"VAROVÁNÍ: Chybí natrénovaný model! Chyba: {e}")
    print("         Ujistěte se, že jste nejprve úspěšně dokončili trénink všech modelů.")


# ==============================================================================
# 1. KONFIGURACE TESTU
# ==============================================================================
TEST_SEQ_LEN = 1000
NUM_TEST_TRAJ = 10
J_SAMPLES_TEST = 25 # (Momentálně se nepoužívá, BKN je zakomentován)

# ==============================================================================
# 2. PŘÍPRAVA DAT
# ==============================================================================
print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií o délce {TEST_SEQ_LEN}...")
x_test, y_test = utils.generate_data(sys_true, num_trajectories=NUM_TEST_TRAJ, seq_len=TEST_SEQ_LEN)
test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print("Generování dat dokončeno.")

# ==============================================================================
# 3. INICIALIZACE VŠECH FILTRŮ PRO POROVNÁNÍ
# ==============================================================================
ekf_mismatched = Filters.ExtendedKalmanFilter(sys_model)
ekf_ideal = Filters.ExtendedKalmanFilter(sys_true)
ukf_mismatched = Filters.UnscentedKalmanFilter(sys_model)
ukf_ideal = Filters.UnscentedKalmanFilter(sys_true)
aekf_mismatched = Filters.AdaptiveExtendedKalmanFilter(sys_model, Q_init=sys_model.Q, R_init=sys_model.R,alpha=0.92)
print("Všechny model-based filtry (EKF, UKF, AEKF) inicializovány.")

# ==============================================================================
# 4. VYHODNOCOVACÍ SMYČKA
# ==============================================================================
all_x_true_cpu = []
# all_x_hat_bkn_cpu, all_P_hat_bkn_cpu = [], [] # BKN je zakomentován
all_x_hat_classic_knet_cpu = []
all_x_hat_knetR_cpu, all_P_hat_knetR_cpu = [], []
all_x_hat_ekf_mismatched_cpu, all_P_hat_ekf_mismatched_cpu = [], []
all_x_hat_ekf_ideal_cpu, all_P_hat_ekf_ideal_cpu = [], []
all_x_hat_ukf_mismatched_cpu, all_P_hat_ukf_mismatched_cpu = [], []
all_x_hat_ukf_ideal_cpu, all_P_hat_ukf_ideal_cpu = [], []
# all_x_hat_aekf_mismatched_cpu, all_P_hat_aekf_mismatched_cpu = [], []
all_x_hat_kformer_cpu = [] # TOTO JSTE TAM JIŽ MĚL - SUPER!

print(f"\nVyhodnocuji modely na {NUM_TEST_TRAJ} testovacích trajektoriích...")

# trained_model_bkn.eval() 
trained_model_classic.eval()
# trained_model_knetR.eval()
trained_model_former.eval() # TOTO JSTE TAM JIŽ MĚL - SUPER!

with torch.no_grad():
    for i, (x_true_seq_batch, y_test_seq_batch) in enumerate(test_loader):
        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)
        
        # --- A. Bayesian KalmanNet (Trajectory-wise) ---
        # (Zakomentováno)

        # --- B. Klasický StateKalmanNet (pouze MSE) ---
        trained_model_classic.reset(batch_size=1, initial_state=initial_state)
        classic_knet_preds = []
        for t in range(1, TEST_SEQ_LEN):
            x_filtered_t = trained_model_classic.step(y_test_seq_gpu[t, :].unsqueeze(0))
            classic_knet_preds.append(x_filtered_t)
        full_x_hat_classic_knet = torch.cat([initial_state, torch.cat(classic_knet_preds, dim=0)], dim=0)
        
        # --- (Nový) KalmanFormer (pouze MSE) ---
        # TUTO SMYČKU JSTE TAM JIŽ MĚL - PERFEKTNÍ!
        trained_model_former.reset(batch_size=1, initial_state=initial_state)
        kformer_preds = []
        for t in range(1, TEST_SEQ_LEN):
            x_filtered_t = trained_model_former.step(y_test_seq_gpu[t, :].unsqueeze(0))
            kformer_preds.append(x_filtered_t)
        full_x_hat_kformer = torch.cat([initial_state, torch.cat(kformer_preds, dim=0)], dim=0)
        
        # # --- C. StateKalmanNetWithKnownR ---
        # trained_model_knetR.reset(batch_size=1, initial_state=initial_state)
        # knetR_preds_x, knetR_preds_P = [], []
        # for t in range(1, TEST_SEQ_LEN):
        #     x_filtered_t, P_filtered_t = trained_model_knetR.step(y_test_seq_gpu[t, :].unsqueeze(0))
        #     knetR_preds_x.append(x_filtered_t)
        #     knetR_preds_P.append(P_filtered_t)
        # full_x_hat_knetR = torch.cat([initial_state, torch.cat(knetR_preds_x, dim=0)], dim=0)
        
        # # (Zbytek kódu pro KNetR P... zůstává stejný)
        # processed_P_list = []
        # for p_tensor in knetR_preds_P:
        #     while p_tensor.dim() < 2:
        #         p_tensor = p_tensor.unsqueeze(-1)
        #     if p_tensor.dim() > 2 and p_tensor.shape[0] == 1:
        #         p_tensor = p_tensor.squeeze(0)
        #     processed_P_list.append(p_tensor)
        # P_sequence_knetR = torch.stack(processed_P_list, dim=0)
        # P0_for_cat = sys_model.P0.clone()
        # while P0_for_cat.dim() < P_sequence_knetR.dim():
        #     P0_for_cat = P0_for_cat.unsqueeze(0)
        # full_P_hat_knetR = torch.cat([P0_for_cat, P_sequence_knetR], dim=0)

        # --- D. EKF (nepřesný a ideální) ---
        ekf_m_res = ekf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
        full_x_hat_ekf_m = ekf_m_res['x_filtered']
        full_P_hat_ekf_m = ekf_m_res['P_filtered']
        ekf_i_res = ekf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)
        full_x_hat_ekf_i = ekf_i_res['x_filtered']
        full_P_hat_ekf_i = ekf_i_res['P_filtered']

        # --- E. UKF (nepřesný a ideální) ---
        ukf_m_res = ukf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
        full_x_hat_ukf_m = ukf_m_res['x_filtered']
        full_P_hat_ukf_m = ukf_m_res['P_filtered']
        ukf_i_res = ukf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)
        full_x_hat_ukf_i = ukf_i_res['x_filtered']
        full_P_hat_ukf_i = ukf_i_res['P_filtered']

        # # --- F. Adaptivní EKF (nepřesný) ---
        # aekf_m_res = aekf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
        # full_x_hat_aekf_m = aekf_m_res['x_filtered']
        # full_P_hat_aekf_m = aekf_m_res['P_filtered']
        
        # --- G. Uložení všech výsledků na CPU ---
        all_x_true_cpu.append(x_true_seq_gpu.cpu())
        # all_x_hat_bkn_cpu.append(full_x_hat_bkn.cpu()); all_P_hat_bkn_cpu.append(full_P_hat_bkn.cpu())
        all_x_hat_classic_knet_cpu.append(full_x_hat_classic_knet.cpu())
        # all_x_hat_knetR_cpu.append(full_x_hat_knetR.cpu()); all_P_hat_knetR_cpu.append(full_P_hat_knetR.cpu())
        all_x_hat_ekf_mismatched_cpu.append(full_x_hat_ekf_m.cpu()); all_P_hat_ekf_mismatched_cpu.append(full_P_hat_ekf_m.cpu())
        all_x_hat_ekf_ideal_cpu.append(full_x_hat_ekf_i.cpu()); all_P_hat_ekf_ideal_cpu.append(full_P_hat_ekf_i.cpu())
        all_x_hat_ukf_mismatched_cpu.append(full_x_hat_ukf_m.cpu()); all_P_hat_ukf_mismatched_cpu.append(full_P_hat_ukf_m.cpu())
        all_x_hat_ukf_ideal_cpu.append(full_x_hat_ukf_i.cpu()); all_P_hat_ukf_ideal_cpu.append(full_P_hat_ukf_i.cpu())
        # all_x_hat_aekf_mismatched_cpu.append(full_x_hat_aekf_m.cpu()); all_P_hat_aekf_mismatched_cpu.append(full_P_hat_aekf_m.cpu())
        
        all_x_hat_kformer_cpu.append(full_x_hat_kformer.cpu()) # <--- 1. PŘIDÁNO: Uložení výsledku KFormer

        print(f"Dokončena trajektorie {i + 1}/{NUM_TEST_TRAJ}...")

# ==============================================================================
# 5. FINÁLNÍ VÝPOČET A VÝPIS METRIK
# ==============================================================================
# Seznamy pro sběr metrik
# mse_bkn, anees_bkn = [], [];
mse_classic_knet = []; mse_knetR, anees_knetR = [], []
mse_ekf_mis, anees_ekf_mis = [], []; mse_ekf_ideal, anees_ekf_ideal = [], []
mse_ukf_mis, anees_ukf_mis = [], []; mse_ukf_ideal, anees_ukf_ideal = [], []
mse_aekf_mis, anees_aekf_mis = [], []
mse_kformer = [] # <--- 2. PŘIDÁNO: Seznam pro MSE KalmanFormeru

print("\nPočítám finální metriky pro jednotlivé trajektorie...")

with torch.no_grad():
    for i in range(NUM_TEST_TRAJ):
        x_true = all_x_true_cpu[i]
        def get_metrics(x_hat, P_hat):
            mse = F.mse_loss(x_hat[1:], x_true[1:]).item()
            # Musíme zajistit, aby P_hat mělo správný tvar (Batch, Seq, Dim, Dim)
            # ANEES se počítá od t=1 (proto [1:])
            P_hat_seq = P_hat[1:] 
            if P_hat_seq.dim() == 3: # Chybí dimenze dávky
                 P_hat_seq = P_hat_seq.unsqueeze(0)
            
            anees = utils.calculate_anees_vectorized(
                x_true[1:].unsqueeze(0), 
                x_hat[1:].unsqueeze(0), 
                P_hat_seq
            )
            return mse, anees

        # Výpočty pro všechny modely
        # mse, anees = get_metrics(all_x_hat_bkn_cpu[i], all_P_hat_bkn_cpu[i]); mse_bkn.append(mse); anees_bkn.append(anees)
        mse = F.mse_loss(all_x_hat_classic_knet_cpu[i][1:], x_true[1:]).item(); mse_classic_knet.append(mse)
        # mse, anees = get_metrics(all_x_hat_knetR_cpu[i], all_P_hat_knetR_cpu[i]); mse_knetR.append(mse); anees_knetR.append(anees)
        mse, anees = get_metrics(all_x_hat_ekf_mismatched_cpu[i], all_P_hat_ekf_mismatched_cpu[i]); mse_ekf_mis.append(mse); anees_ekf_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_ekf_ideal_cpu[i], all_P_hat_ekf_ideal_cpu[i]); mse_ekf_ideal.append(mse); anees_ekf_ideal.append(anees)
        mse, anees = get_metrics(all_x_hat_ukf_mismatched_cpu[i], all_P_hat_ukf_mismatched_cpu[i]); mse_ukf_mis.append(mse); anees_ukf_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_ukf_ideal_cpu[i], all_P_hat_ukf_ideal_cpu[i]); mse_ukf_ideal.append(mse); anees_ukf_ideal.append(anees)
        # mse, anees = get_metrics(all_x_hat_aekf_mismatched_cpu[i], all_P_hat_aekf_mismatched_cpu[i]); mse_aekf_mis.append(mse); anees_aekf_mis.append(anees)

        # <--- 3. PŘIDÁNO: Výpočet MSE pro KalmanFormer ---
        mse_kf = F.mse_loss(all_x_hat_kformer_cpu[i][1:], x_true[1:]).item()
        mse_kformer.append(mse_kf)


# Funkce pro bezpečné průměrování
def avg(metric_list): return np.mean([m for m in metric_list if not np.isnan(m)])
state_dim_for_nees = all_x_true_cpu[0].shape[1]

# --- Finální výpis tabulky ---
print("\n" + "="*80)
print(f"FINÁLNÍ VÝSLEDKY (průměr přes {NUM_TEST_TRAJ} běhů)")
print("="*80)
print(f"{'Model':<35} | {'Průměrné MSE':<20} | {'Průměrný ANEES':<20}")
print("-" * 80)
print(f"{'--- Data-Driven Models ---':<35} | {'(nižší je lepší)':<20} | {'(bližší ' + str(float(state_dim_for_nees)) + ' je lepší)':<20}")
# print(f"{'Bayesian KNet (BKN)':<35} | {avg(mse_bkn):<20.4f} | {avg(anees_bkn):<20.4f}")
print(f"{'KNet (pouze MSE)':<35} | {avg(mse_classic_knet):<20.4f} | {'N/A':<20}")
print(f"{'KalmanFormer (pouze MSE)':<35} | {avg(mse_kformer):<20.4f} | {'N/A':<20}") # <--- 4. PŘIDÁNO: Řádek v tabulce
# print(f"{'KNet with Known R (KNetR)':<35} | {avg(mse_knetR):<20.4f} | {avg(anees_knetR):<20.4f}")
print("-" * 80)
print(f"{'--- Model-Based Filters ---':<35} | {'':<20} | {'':<20}")
print(f"{'EKF (Nepřesný model)':<35} | {avg(mse_ekf_mis):<20.4f} | {avg(anees_ekf_mis):<20.4f}")
print(f"{'UKF (Nepřesný model)':<35} | {avg(mse_ukf_mis):<20.4f} | {avg(anees_ukf_mis):<20.4f}")
# print(f"{'AEKF (Nepřesný model)':<35} | {avg(mse_aekf_mis):<20.4f} | {avg(anees_aekf_mis):<20.4f}")
print("-" * 80)
print(f"{'--- Benchmarks ---':<35} | {'':<20} | {'':<20}")
print(f"{'EKF (Ideální model)':<35} | {avg(mse_ekf_ideal):<20.4f} | {avg(anees_ekf_ideal):<20.4f}")
print(f"{'UKF (Ideální model)':<35} | {avg(mse_ukf_ideal):<20.4f} | {avg(anees_ukf_ideal):<20.4f}")
print("="*80)

INFO: Všechny natrénované modely nalezeny a přiřazeny.

Generuji 10 testovacích trajektorií o délce 1000...
Generování dat dokončeno.
Všechny model-based filtry (EKF, UKF, AEKF) inicializovány.

Vyhodnocuji modely na 10 testovacích trajektoriích...
Dokončena trajektorie 1/10...
Dokončena trajektorie 2/10...
Dokončena trajektorie 3/10...
Dokončena trajektorie 4/10...
Dokončena trajektorie 5/10...
Dokončena trajektorie 6/10...
Dokončena trajektorie 7/10...
Dokončena trajektorie 8/10...
Dokončena trajektorie 9/10...
Dokončena trajektorie 10/10...

Počítám finální metriky pro jednotlivé trajektorie...

FINÁLNÍ VÝSLEDKY (průměr přes 10 běhů)
Model                               | Průměrné MSE         | Průměrný ANEES      
--------------------------------------------------------------------------------
--- Data-Driven Models ---          | (nižší je lepší)     | (bližší 2.0 je lepší)
KNet (pouze MSE)                    | 0.6460               | N/A                 
KalmanFormer (pouze MSE)   